<a href="https://colab.research.google.com/github/Tomiinek/Multilingual_Text_to_Speech/blob/master/notebooks/multi_training_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual Text-to-Speech Demo

This notebook demonstrates multilingual text-to-speech (without voice cloning) using:

- Tacotron based spectrogram generation: https://github.com/Tomiinek/Multilingual_Text_to_Speech
- WaveRNN vocoder: https://github.com/Tomiinek/WaveRNN, forked from fatchord/WaveRNN


**Estimated time to complete**: 4 minutes



In [0]:
import sys
import os
import IPython
from IPython.display import Audio

## Clone repositories

In [0]:
os.chdir(os.path.expanduser("~"))
    
tacotron_dir = "Multilingual_Text_to_Speech"
if not os.path.exists(tacotron_dir):
  ! git clone https://github.com/Tomiinek/Multilingual_Text_to_Speech # $tacotron_dir

wavernn_dir = "WaveRNN"
if not os.path.exists(wavernn_dir):
  ! git clone https://github.com/Tomiinek/$wavernn_dir

## Download pre-trained models

In [0]:
! mkdir -p checkpoints
os.chdir(os.path.join(os.path.expanduser("~"), "checkpoints"))

tacotron_chpt = "generated_training.pyt"
if not os.path.exists(os.path.join(os.path.expanduser("~"), "checkpoints", tacotron_chpt)):
  ! curl -O -L "https://github.com/Tomiinek/Multilingual_Text_to_Speech/releases/download/v1.0/$tacotron_chpt"  

wavernn_chpt = "wavernn_weight.pyt"
if not os.path.exists(os.path.join(os.path.expanduser("~"), "checkpoints", wavernn_chpt)):
  ! curl -O -L "https://github.com/Tomiinek/Multilingual_Text_to_Speech/releases/download/v1.0/$wavernn_chpt"     

os.chdir(os.path.expanduser("~"))

## Install dependencies

In [0]:
! pip install -q -U soundfile
! pip install -q -U phonemizer
! pip install -q -U epitran

## Input texts to be synthesized

Inputs consist of **two parts delimited** by `|`:
  - **Input utterance** - Only a basic normalization is applied to input utterances, so **you should not use obscure characters and punctuation**. See examples that are formatted properly.
  - **Language or Speaker ID** - There are more available language IDs, but **you should use just one of** `spanish` (M), `french` (M), `german` (F), `greek` (F), `dutch` (M), `finnish` (M), `hungarian` (F), `chinese` (F), `japanese` (M), and `russian` (M).

Feel free to modify the examples below.






In [0]:
inputs = [
          "Las hojas con los cantos afilados cortan los objetos que se sitúan entre ellas.|spanish",
          "C'est l'un des plus beaux palais de Mistretta dont le nom dérive d'une ancienne famille seigneuriale de la ville.|french",
          "Nach seiner Rückkehr wurde er Vorstand der Abteilung für Landesaufnahme im sächsischen Generalstab.|german",
          "Αυτές οι κριτικές αναζητήσεις εμφανίζονται στην ελληνική φωτογραφία μόλις στα τέλη της δεκαετίας.|greek",
          "Soms is dit inderdaad een papieren bonnetje, maar vaak ook een plastic muntje of een metalen munt.|dutch",
          "Päälaivojen korkeus antoi tilaa klerestorioikkunoille sivulaivojen yläpuolelle.|finnish",  
          "Háborús felek között általánosan elismerten a megadás vagy a fegyverszünet jeleként ismert a legtöbbek számára.|hungarian",
          "jìsuànjī dàxué zhǔyào xuékē shì kēxué hé jìzhúbù， xuéshēng kěyǐ huòqǔ jìsuànjīkēxué hé jìzhú de běnkē xuéwèi。|chinese",
          "yokuasa、 saheiji ha riyuu wo tsuke te jibun ha mou ippaku suru mune wo nakama ni tsuge、 mina wo kaeshi te shimau。|japanese",
          "Из города к церкви по склону холма ведут роскошно декорированная лестница в стиле необарокко.|russian",         
]

## Synthesis

### Spectrogram generation

In [0]:
os.chdir(os.path.join(os.path.expanduser("~"), tacotron_dir))
if "utils" in sys.modules: del sys.modules["utils"]

from synthesize import synthesize
from utils import build_model

model = build_model(os.path.join(os.path.expanduser("~"), "checkpoints", tacotron_chpt))
model.eval()

spectrograms = []
for i in inputs:
  tokens = i.split("|")
  s = synthesize(model, "|" + tokens[0] + "|" + tokens[1] + "|" + tokens[1])
  spectrograms.append(s)

### Waveform generation

In [0]:
os.chdir(os.path.join(os.path.expanduser("~"), wavernn_dir))
if "utils" in sys.modules: del sys.modules["utils"]

from models.fatchord_version import WaveRNN
from utils import hparams as hp
from gen_wavernn import generate
import torch

hp.configure('hparams.py')
model = WaveRNN(rnn_dims=hp.voc_rnn_dims, fc_dims=hp.voc_fc_dims, bits=hp.bits, pad=hp.voc_pad, upsample_factors=hp.voc_upsample_factors, 
                feat_dims=hp.num_mels, compute_dims=hp.voc_compute_dims, res_out_dims=hp.voc_res_out_dims, res_blocks=hp.voc_res_blocks, 
                hop_length=hp.hop_length, sample_rate=hp.sample_rate, mode=hp.voc_mode).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model.load(os.path.join(os.path.expanduser("~"), "checkpoints", wavernn_chpt))

waveforms = [generate(model, s, hp.voc_gen_batched, hp.voc_target, hp.voc_overlap) for s in spectrograms]

## Resulting audios

In [0]:
for idx, w in enumerate(waveforms):
  print(inputs[idx])
  IPython.display.display(IPython.display.Audio(data=w, rate=hp.sample_rate))